In [37]:
import pandas as pd
import numpy as np

# Config: paths
SRC      = "data/matches_with_stats_cleaned.csv"     # original merged file
OUT_RAW  = "data/matches_ml_prematch_raw.csv"        # intermediate (optional)
OUT_CLEAN= "data/matches_ml_prematch_clean.csv"      # final ML dataset

# 1) Load only columns that exist
df = pd.read_csv(SRC, parse_dates=["tourney_date"])

# pre‑match columns we know (drop in‑match stats entirely)
ctx_cols = ["surface","round","tourney_level","best_of","draw_size","tourney_date"]
A_cols  = ["winner_rank","winner_seed","winner_hand","winner_age","winner_ht"]
B_cols  = ["loser_rank","loser_seed","loser_hand","loser_age","loser_ht"]
have_ctx= [c for c in ctx_cols if c in df.columns]
have_A  = [c for c in A_cols  if c in df.columns]
have_B  = [c for c in B_cols  if c in df.columns]

df = df[have_ctx + have_A + have_B].copy().reset_index(drop=True)
df["year"]    = df["tourney_date"].dt.year
df["match_id"]= df.index

# 2) Convert each original match to a single A/B row with random orientation
def make_ab_row(row, flip=False):
    g = row.get
    if not flip:
        A = dict(rank=g("winner_rank"), seed=g("winner_seed"),
                 hand=g("winner_hand"), age=g("winner_age"), ht=g("winner_ht"))
        B = dict(rank=g("loser_rank"),  seed=g("loser_seed"),
                 hand=g("loser_hand"),  age=g("loser_age"),  ht=g("loser_ht"))
        label = 1
    else:
        A = dict(rank=g("loser_rank"),  seed=g("loser_seed"),
                 hand=g("loser_hand"),  age=g("loser_age"),  ht=g("loser_ht"))
        B = dict(rank=g("winner_rank"), seed=g("winner_seed"),
                 hand=g("winner_hand"), age=g("winner_age"), ht=g("winner_ht"))
        label = 0
    return {
        "surface"     : g("surface"),
        "round"       : g("round"),
        "tourney_level": g("tourney_level"),
        "best_of"     : g("best_of"),
        "draw_size"   : g("draw_size"),
        "year"        : g("year"),
        "match_id"    : g("match_id"),
        # A/B player meta (no seeds/ages/heights/hands for leakage)
        "playerA_rank": A["rank"],
        "playerB_rank": B["rank"],
        # target
        "label"       : label,
    }

# random flip mask
rng = np.random.default_rng(42)
flip_mask = rng.random(len(df)) < 0.5
rows = [make_ab_row(row, flip=bool(flip_mask[i])) for i, row in df.iterrows()]
prematch = pd.DataFrame(rows)

# 3) Feature engineering: rank diffs and logs
prematch["rank_diff"]       = prematch["playerA_rank"] - prematch["playerB_rank"]
prematch["log_playerA_rank"]= np.log1p(prematch["playerA_rank"])
prematch["log_playerB_rank"]= np.log1p(prematch["playerB_rank"])
prematch["log_rank_diff"]   = prematch["log_playerA_rank"] - prematch["log_playerB_rank"]

# Round code (progress order) – optional but useful
round_order = {"R128":1,"R64":2,"R32":3,"R16":4,"QF":5,"SF":6,"F":7}
prematch["round_code"] = prematch["round"].map(round_order)

# 4) Save raw engineered file (optional)
prematch.to_csv(OUT_RAW, index=False)

# 5) Clean NaNs on critical columns (surface, level, round, best_of, year, ranks)
critical_cols = ["surface","tourney_level","round","best_of","year",
                 "playerA_rank","playerB_rank"]
prematch_clean = prematch.dropna(subset=critical_cols).reset_index(drop=True)

# Fill remaining NaNs (e.g., draw_size) with per‑year median
if "draw_size" in prematch_clean.columns:
    med_by_year = prematch_clean.groupby("year")["draw_size"].transform("median")
    prematch_clean["draw_size"] = prematch_clean["draw_size"].fillna(med_by_year)
    prematch_clean["draw_size"] = prematch_clean["draw_size"].fillna(prematch_clean["draw_size"].median())

# (Optional) remove any remaining NaNs in numeric features
feature_cols = [c for c in prematch_clean.columns if c not in ["label","match_id"]]
prematch_clean = prematch_clean.dropna(subset=feature_cols).reset_index(drop=True)

# 6) Save final cleaned dataset
prematch_clean.to_csv(OUT_CLEAN, index=False)
print(f" Pre‑match dataset saved to: {OUT_CLEAN} (rows: {len(prematch_clean)})")


 Pre‑match dataset saved to: data/matches_ml_prematch_clean.csv (rows: 42703)


In [38]:
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)

prematch_clean.sample(15, random_state=42)


,surface,round,tourney_level,best_of,draw_size,year,match_id,playerA_rank,playerB_rank,label,rank_diff,log_playerA_rank,log_playerB_rank,log_rank_diff,round_code
4543,Carpet,R32,A,3,32.0,1970,4543,47.0,34.0,0,13.0,3.871201,3.555348,0.315853,3.0
23850,Hard,R32,A,3,32.0,1970,23887,355.0,10.0,0,345.0,5.874931,2.397895,3.477035,3.0
2746,Hard,SF,A,3,32.0,1970,2746,187.0,60.0,0,127.0,5.236442,4.110874,1.125568,6.0
27704,Hard,R32,G,5,128.0,1970,27741,107.0,6.0,0,101.0,4.682131,1.945910,2.736221,3.0
6976,Hard,QF,A,3,32.0,1970,6976,74.0,193.0,1,-119.0,4.317488,5.267858,-0.950370,5.0
19286,Hard,SF,A,3,32.0,1970,19286,22.0,30.0,1,-8.0,3.135494,3.433987,-0.298493,6.0
17036,Hard,R32,A,3,32.0,1970,17036,10.0,56.0,1,-46.0,2.397895,4.043051,-1.645156,3.0
41449,Hard,SF,A,3,32.0,1970,41724,39.0,8.0,0,31.0,3.688879,2.197225,1.491655,6.0
10115,Hard,QF,A,3,48.0,1970,10115,43.0,5.0,0,38.0,3.784190,1.791759,1.992430,5.0
16028,Clay,F,A,3,32.0,1970,16028,6.0,36.0,1,-30.0,1.945910,3.610918,-1.665008,7.0
